In [ ]:
from google.colab import files
uploaded = files.upload()


Saving cleaned_df.csv to cleaned_df.csv


In [ ]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [59]:
from numpy import array
from numpy import asarray
from numpy import zeros
import tensorflow.keras as keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
import re, os, sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, Dense, Input, Flatten, Conv1D, MaxPooling1D, Dropout, BatchNormalization, Activation, Add, Concatenate, LSTM
from keras.models import Model
import pickle
import pandas as pd
from keras import backend as K
import string
from nltk.corpus import stopwords
## Plotly
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
# Others
import nltk
from nltk.stem.snowball import SnowballStemmer

In [ ]:
df = pd.read_csv("cleaned_df.csv")

In [ ]:
df.head()

,Unnamed: 0,type,posts,E-I,N-S,F-T,J-P,cleaned_post,words_count,bigrams,trigrams
0,0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...,I,N,F,J,and intj moments sportscenter not top ten play...,454,"[('intj', 'moments'), ('moments', 'sportscente...","[('intj', 'moments', 'sportscenter'), ('moment..."
1,1,ENTP,'I'm finding the lack of me in these posts ver...,E,N,T,P,finding the lack these posts very alarming sex...,874,"[('finding', 'lack'), ('lack', 'posts'), ('pos...","[('finding', 'lack', 'posts'), ('lack', 'posts..."
2,2,INTP,'Good one _____ https://www.youtube.com/wat...,I,N,T,P,good one course which say know that blessing a...,653,"[('good', 'one'), ('one', 'course'), ('course'...","[('good', 'one', 'course'), ('one', 'course', ..."
3,3,INTJ,"'Dear INTP, I enjoyed our conversation the o...",I,N,T,J,dear intp enjoyed our conversation the other d...,820,"[('dear', 'intp'), ('intp', 'enjoyed'), ('enjo...","[('dear', 'intp', 'enjoyed'), ('intp', 'enjoye..."
4,4,ENTJ,'You're fired.|||That's another silly misconce...,E,N,T,J,you are fired that another silly misconception...,782,"[('fired', 'another'), ('another', 'silly'), (...","[('fired', 'another', 'silly'), ('another', 's..."


In [ ]:
training_data = df[["cleaned_post", "E-I", "N-S", "F-T", "J-P"]].copy()
training_data.head(5)

,cleaned_post,E-I,N-S,F-T,J-P
0,and intj moments sportscenter not top ten play...,I,N,F,J
1,finding the lack these posts very alarming sex...,E,N,T,P
2,good one course which say know that blessing a...,I,N,T,P
3,dear intp enjoyed our conversation the other d...,I,N,T,J
4,you are fired that another silly misconception...,E,N,T,J


In [ ]:
def make_dummies(data, columns=["E-I", "N-S", "F-T", "J-P"]):
    for column in columns:
        temp_dummy = pd.get_dummies(data[column])
        data = data.join(temp_dummy)
    return data

In [ ]:
training_data = make_dummies(training_data)
training_data.head()

,cleaned_post,E-I,N-S,F-T,J-P,E,I,N,S,F,T,J,P
0,and intj moments sportscenter not top ten play...,I,N,F,J,0,1,1,0,1,0,1,0
1,finding the lack these posts very alarming sex...,E,N,T,P,1,0,1,0,0,1,0,1
2,good one course which say know that blessing a...,I,N,T,P,0,1,1,0,0,1,0,1
3,dear intp enjoyed our conversation the other d...,I,N,T,J,0,1,1,0,0,1,1,0
4,you are fired that another silly misconception...,E,N,T,J,1,0,1,0,0,1,1,0


In [46]:
y_ei = np.array(y["E"])
y_ns = np.array(y["N"])
y_ft = np.array(y["F"])
y_jp = np.array(y["J"])


In [55]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [60]:
stemmer = SnowballStemmer('english')

In [61]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    ## Stemming
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text

In [62]:
training_data['cleaned_post'] = training_data['cleaned_post'].map(lambda x: clean_text(x))

In [65]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(training_data['cleaned_post'])
sequences = tokenizer.texts_to_sequences(training_data['cleaned_post'])
data = pad_sequences(sequences, maxlen=50)

In [68]:
## Network architecture for TypeE/I
model_ei = Sequential()
model_ei.add(Embedding(20000, 100, input_length=50))
model_ei.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_ei.add(Dense(1, activation='sigmoid'))
model_ei.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model_ei.fit(data, y_ei, validation_split=0.4, epochs=10)

Epoch 1/10
163/163 [==============================] - 26s 143ms/step - loss: 0.5541 - accuracy: 0.7658 - val_loss: 0.5225 - val_accuracy: 0.7735
Epoch 2/10
163/163 [==============================] - 23s 140ms/step - loss: 0.4285 - accuracy: 0.8085 - val_loss: 0.5277 - val_accuracy: 0.7746
Epoch 3/10
163/163 [==============================] - 24s 146ms/step - loss: 0.2474 - accuracy: 0.9064 - val_loss: 0.7016 - val_accuracy: 0.7706
Epoch 4/10
163/163 [==============================] - 24s 146ms/step - loss: 0.1263 - accuracy: 0.9575 - val_loss: 0.7837 - val_accuracy: 0.7403
Epoch 5/10
163/163 [==============================] - 26s 157ms/step - loss: 0.0689 - accuracy: 0.9775 - val_loss: 1.0408 - val_accuracy: 0.7210
Epoch 6/10
163/163 [==============================] - 23s 140ms/step - loss: 0.0466 - accuracy: 0.9854 - val_loss: 1.1492 - val_accuracy: 0.6951
Epoch 7/10
163/163 [==============================] - 23s 140ms/step - loss: 0.0337 - accuracy: 0.9898 - val_loss: 1.4314 - val_ac

In [69]:
## Network architecture for type N/S
model_ns = Sequential()
model_ns.add(Embedding(20000, 100, input_length=50))
model_ns.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_ns.add(Dense(1, activation='sigmoid'))
model_ns.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model_ns.fit(data, y_ns, validation_split=0.4, epochs=10)

Epoch 1/10
163/163 [==============================] - 36s 202ms/step - loss: 0.4173 - accuracy: 0.8622 - val_loss: 0.4090 - val_accuracy: 0.8591
Epoch 2/10
163/163 [==============================] - 29s 176ms/step - loss: 0.3352 - accuracy: 0.8695 - val_loss: 0.4092 - val_accuracy: 0.8501
Epoch 3/10
163/163 [==============================] - 23s 140ms/step - loss: 0.1908 - accuracy: 0.9287 - val_loss: 0.4705 - val_accuracy: 0.8412
Epoch 4/10
163/163 [==============================] - 28s 170ms/step - loss: 0.0945 - accuracy: 0.9702 - val_loss: 0.5950 - val_accuracy: 0.8023
Epoch 5/10
163/163 [==============================] - 26s 158ms/step - loss: 0.0474 - accuracy: 0.9852 - val_loss: 0.7531 - val_accuracy: 0.8303
Epoch 6/10
163/163 [==============================] - 26s 162ms/step - loss: 0.0248 - accuracy: 0.9935 - val_loss: 0.8491 - val_accuracy: 0.8225
Epoch 7/10
163/163 [==============================] - 26s 158ms/step - loss: 0.0127 - accuracy: 0.9971 - val_loss: 0.8809 - val_ac

In [70]:
## Network architecture
model_ft = Sequential()
model_ft.add(Embedding(20000, 100, input_length=50))
model_ft.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_ft.add(Dense(1, activation='sigmoid'))
model_ft.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model_ft.fit(data, y_ft, validation_split=0.4, epochs=10)

Epoch 1/10
163/163 [==============================] - 26s 144ms/step - loss: 0.6631 - accuracy: 0.5919 - val_loss: 0.6274 - val_accuracy: 0.6539
Epoch 2/10
163/163 [==============================] - 23s 141ms/step - loss: 0.4733 - accuracy: 0.7796 - val_loss: 0.6463 - val_accuracy: 0.6516
Epoch 3/10
163/163 [==============================] - 24s 147ms/step - loss: 0.2938 - accuracy: 0.8855 - val_loss: 0.7648 - val_accuracy: 0.6403
Epoch 4/10
163/163 [==============================] - 23s 141ms/step - loss: 0.1479 - accuracy: 0.9481 - val_loss: 1.1230 - val_accuracy: 0.6277
Epoch 5/10
163/163 [==============================] - 23s 144ms/step - loss: 0.0662 - accuracy: 0.9783 - val_loss: 1.5457 - val_accuracy: 0.6251
Epoch 6/10
163/163 [==============================] - 26s 158ms/step - loss: 0.0371 - accuracy: 0.9887 - val_loss: 1.7692 - val_accuracy: 0.6069
Epoch 7/10
163/163 [==============================] - 23s 142ms/step - loss: 0.0220 - accuracy: 0.9931 - val_loss: 1.7444 - val_ac

In [71]:
## Network architecture
model_jp = Sequential()
model_jp.add(Embedding(20000, 100, input_length=50))
model_jp.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_jp.add(Dense(1, activation='sigmoid'))
model_jp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
## Fit the model
model_ei.fit(data, y_jp, validation_split=0.4, epochs=10)

Epoch 1/10
163/163 [==============================] - 23s 142ms/step - loss: 0.8042 - accuracy: 0.5629 - val_loss: 0.6698 - val_accuracy: 0.6086
Epoch 2/10
163/163 [==============================] - 23s 141ms/step - loss: 0.6468 - accuracy: 0.6183 - val_loss: 0.6949 - val_accuracy: 0.5620
Epoch 3/10
163/163 [==============================] - 23s 140ms/step - loss: 0.5631 - accuracy: 0.7037 - val_loss: 0.7434 - val_accuracy: 0.5464
Epoch 4/10
163/163 [==============================] - 23s 141ms/step - loss: 0.4186 - accuracy: 0.8050 - val_loss: 0.8827 - val_accuracy: 0.5225
Epoch 5/10
163/163 [==============================] - 26s 157ms/step - loss: 0.2919 - accuracy: 0.8751 - val_loss: 0.9731 - val_accuracy: 0.5651
Epoch 6/10
163/163 [==============================] - 23s 141ms/step - loss: 0.1757 - accuracy: 0.9370 - val_loss: 1.2197 - val_accuracy: 0.5620
Epoch 7/10
163/163 [==============================] - 23s 141ms/step - loss: 0.1021 - accuracy: 0.9646 - val_loss: 1.6447 - val_ac

In [72]:
# Save model
filename = f'LSTM_E-I.sav'
pickle.dump(model_ei,open(filename, 'wb'))

In [73]:
# Save model
filename = f'LSTM_N-S.sav'
pickle.dump(model_ns,open(filename, 'wb'))

In [74]:
# Save model
filename = f'LSTM_F-T.sav'
pickle.dump(model_ft,open(filename, 'wb'))

In [76]:
# Save model
filename = f'LSTM_J-P.sav'
pickle.dump(model_jp,open(filename, 'wb'))

# Conclusion


Using LSTM, the accuracy score is : 
 

*   0.9971 for E/I
*   0.9975 for N/S
*   0.9965 for F/T
*   0.9942 for J/P




